In [1]:
!pip install transformers huggingface_hub 
!pip install "git+https://github.com/Tfloow/auto_adpq.git"


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/Tfloow/auto_adpq.git to c:\users\thoma\appdata\local\temp\pip-req-build-5m8xxtna
  Resolved https://github.com/Tfloow/auto_adpq.git to commit 664eb9ce48e250ed7303a2b6642fd83692be41ef
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/Tfloow/auto_adpq.git 'C:\Users\thoma\AppData\Local\Temp\pip-req-build-5m8xxtna'

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Hugging face hub login
import huggingface_hub
with open("../.env", "r") as f:
    token = f.read().strip().split('=')[1]
huggingface_hub.login(token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [19]:
# Load llama 3.1 8B model and quantize it with Auto-AdpQ
from transformers import AutoModelForCausalLM, AutoTokenizer
from auto_adpq import Auto_AdpQ, AutoAdpQConfig
import os
import glob
import gc

In [18]:
save_path = "../../MasterThesis/experiments/weights/meta-llama/Meta-Llama-3.1-8B-weights" 
# Check if the model is present
files = glob.glob(os.path.join(save_path, "*.safetensors"))
if len(files) == 0:
    model_name = "meta-llama/Llama-3.1-8B"
else:
    model_name = save_path
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cpu", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Setup Auto-AdpQ configuration
adpq_config = AutoAdpQConfig(
    group_size = 128,
    n_iters = 100,
    alpha = 0.08,
    device = "cpu",
    q_bit = 4,
    data_packing = True,
    symmetrical_quantization = True
)

adpq = Auto_AdpQ(adpq_config)

In [ ]:
# Quantize the model
quantized_model = adpq.quantize_model(model)

# Save the quantized model
adpq.save_pretrained(quantized_model, "quantized_meta_llama_3.1_8B_adpq")

In [ ]:
del model
gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()